In [16]:
import chime, warnings,time
chime.notify_exceptions()
warnings.filterwarnings('ignore')

# Required Dependencies for the whole notebook

In [1]:
# Data Science basic imports
import pandas as pd, numpy as np, seaborn as sb, matplotlib.pyplot as plt, matplotlib.dates as mdates
# ML Data Preprocessing imports
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler # Z-Score
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Business Understanding - Data Understanding - Data Preparation - EDA

<img src = 'datasc-660x434.png'>

# Regression Problem

The business problem is understanding the COVID-19.The cumulative count of confirmed, death and recovered cases of COVID-19 from different countries from 22nd January 2020
1. Collecting data https://www.kaggle.com/datasets/imdevskp/corona-virus-report?select=covid_19_clean_complete.csv
- [Note: the downloaded file is in .zip, we need to unzip it then import it as csv]
2. Understanding the data by using the traditional Python data science modules. 

In this notebook, we will walkthrough the pandas library which is used for data wrangling tasks: data assessing, data coding and data cleaning. This is a part of business & data understanding and preparation. [Note: Best practice is always to import os module along with numpy]

- Importing data and dealing with data with different formats: CSV, JSON, TSV, XLSX, TXT, HTML and etc ...
  - pd.read_csv('FILENAME.CSV')
  - pd.read_excel('FILENAME.XLSX',sheet = n) # where n is the sheet number 
- Functions and command to assess the data
  - head(n): Finding n top rows
  - tail(n): Finding n bottom rows
  - sample(n): Finding n sample of data
  - info(): get an overall information about the data attributes/types/missing values
  - shape: finding the shape of data in rows x columns
  - describe(): find statistical measures about your data including: mean, std, Q1,2,3,4 and range, and mode and median.
  - plot(): plotting the data
- Creating a data frame or a data series
  - DataFrame({'COL1_NAME': [Values],'COL2_NAME': [Values]})
  - Series({'INDEX_NAME': [Values]})
- Important functions and cheatsheet check [Cheatsheet](./Pandas_Cheat_Sheet.pdf)
  - lambda function
  - join
  - melt
  - merge
  - concatenate
  and etc ...

## Wrangling Process [Data Understanding & Data Preparation]
- Importing the data and the required modules
- Assessing the data
- Documenting the observations
- Taking action and decision regarding wrangling the data

In [78]:
# Hint use pd.read_csv(filename, compression = 'zip')
# file name is covid_19_clean_complete.csv.zip
# Unzipping the data file since it is a zip file we add attribute compression = 'zip'
...

In [79]:
pd.set_option('display.max_rows',None,'display.max_colwidth',None,'display.max_columns',None)

In [80]:
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  14664 non-null  object 
 1   Country/Region  49068 non-null  object 
 2   Lat             49068 non-null  float64
 3   Long            49068 non-null  float64
 4   Date            49068 non-null  object 
 5   Confirmed       49068 non-null  int64  
 6   Deaths          49068 non-null  int64  
 7   Recovered       49068 non-null  int64  
 8   Active          49068 non-null  int64  
 9   WHO Region      49068 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 3.7+ MB


In [2]:
# Hint: Find duplicates if any
...

In [83]:
df.shape

(49068, 10)

In [84]:
df.describe()

,Lat,Long,Confirmed,Deaths,Recovered,Active
count,49068.000000,49068.000000,4.906800e+04,49068.000000,4.906800e+04,4.906800e+04
mean,21.433730,23.528236,1.688490e+04,884.179160,7.915713e+03,8.085012e+03
std,24.950320,70.442740,1.273002e+05,6313.584411,5.480092e+04,7.625890e+04
min,-51.796300,-135.000000,0.000000e+00,0.000000,0.000000e+00,-1.400000e+01
25%,7.873054,-15.310100,4.000000e+00,0.000000,0.000000e+00,0.000000e+00
50%,23.634500,21.745300,1.680000e+02,2.000000,2.900000e+01,2.600000e+01
75%,41.204380,80.771797,1.518250e+03,30.000000,6.660000e+02,6.060000e+02
max,71.706900,178.065000,4.290259e+06,148011.000000,1.846641e+06,2.816444e+06


Observations - Hint: What do you see?:
1. Obs1: 
2. Obs2: 

Strategy to solve the observation if any issues:
1. Solution1: 
2. Solution2: 

## Exploratory Data Analysis [EDA]

In [3]:
# Hint: Do some plots after cleaning data and provide some insights
# Hint: Ask yourself a question before answering it using graph.

Graph Observations:

1. Obs1: 
2. Obs2: 


# Data Modelling [Machine Learning Model]

<img src = basic_modelling1.png>